In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = '/content/drive/MyDrive/전력수요예측/'

In [3]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# train 데이터 불러오기 (더미 o)

In [4]:
# train 파일 불러오고 전처리
train = pd.read_csv(path+'electric_train_cp949.csv')
train.drop('Unnamed: 0', axis=1, inplace=True)
cols = [col[(col.find('.')+1):] for col in train.columns]
train.columns = cols

# datetime 형식으로 변환
train['tm'] = pd.to_datetime(train['tm'])

# 컬럼 추가
train['year'] = train['tm'].dt.year
train['month'] = train['tm'].dt.month
train['day'] = train['tm'].dt.day

# month 더미화
train = pd.get_dummies(train, columns=['hh24','month'], drop_first=True)


# train 데이터의 elec 이상치 처리
for t in list(train.loc[train['elec'] < 0, 'nph_ta']):
    train.loc[(train['elec'] < 0)&(train['nph_ta'] == t), 'elec']\
    = train.loc[(train['nph_ta'] == t)&(train['elec'] >= 0), :]['elec'].mean()

In [7]:
# test 파일 불러오고 전처리
test = pd.read_csv(path+'electric_test.csv')
test.columns = ['num', 'tm', 'hh24', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m',
       'nph_rn_60m', 'nph_ta_chi', 'weekday', 'week_name', 'elect']
test.drop(['elect'] ,axis=1, inplace=True)

# datetime 형식으로 변환
test['tm'] = pd.to_datetime(test['tm'])

# 컬럼 추가
test['year'] = test['tm'].dt.year
test['month'] = test['tm'].dt.month
test['day'] = test['tm'].dt.day
test.drop('tm', axis=1, inplace=True)

# month 더미화
test = pd.get_dummies(test, columns=['hh24','month'], drop_first=True)

In [5]:
X = train.drop(['tm', 'n', 'n_mean_load', 'sum_qctr', 'sum_load', 'elec'], axis=1)
Y = train.loc[:,['elec']]

print(X.columns)
# print(test.columns)

Index(['num', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m',
       'nph_ta_chi', 'weekday', 'week_name', 'year', 'day', 'hh24_2', 'hh24_3',
       'hh24_4', 'hh24_5', 'hh24_6', 'hh24_7', 'hh24_8', 'hh24_9', 'hh24_10',
       'hh24_11', 'hh24_12', 'hh24_13', 'hh24_14', 'hh24_15', 'hh24_16',
       'hh24_17', 'hh24_18', 'hh24_19', 'hh24_20', 'hh24_21', 'hh24_22',
       'hh24_23', 'hh24_24', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12'],
      dtype='object')


In [6]:
from sklearn.preprocessing import MinMaxScaler
X_cols = X.columns
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X =pd.DataFrame(X)
X.columns = X_cols

# test_cols = test.columns
# test = scaler.transform(test)
# test =pd.DataFrame(test)
# test.columns = test_cols

print()
y_max, y_min = Y.max(), Y.min()
Y = (Y-y_min) / (y_max-y_min)

print(y_max)
print(y_min)


elec    373.06
dtype: float64
elec    48.9
dtype: float64


In [7]:
train = pd.merge(X,Y,left_index=True, right_index=True)

In [8]:
del X, Y

In [9]:
display(train.head())
# display(test.head())

,num,stn,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,year,...,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,elec
0,0.0,0.880179,0.435772,0.616650,0.797468,0.0,0.400936,0.666667,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.156281
1,0.0,0.880179,0.437398,0.620761,0.799842,0.0,0.407176,0.666667,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132280
2,0.0,0.880179,0.435772,0.613566,0.803006,0.0,0.396256,0.666667,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093380
3,0.0,0.880179,0.427642,0.624872,0.796677,0.0,0.413417,0.666667,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.075395
4,0.0,0.880179,0.427642,0.619733,0.795886,0.0,0.404056,0.666667,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.064258


## timesteps = 36

In [10]:
timesteps = 24

x_cols = ['num', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m',
       'nph_ta_chi', 'weekday', 'week_name', 'year', 'day', 'hh24_2', 'hh24_3',
       'hh24_4', 'hh24_5', 'hh24_6', 'hh24_7', 'hh24_8', 'hh24_9', 'hh24_10',
       'hh24_11', 'hh24_12', 'hh24_13', 'hh24_14', 'hh24_15', 'hh24_16',
       'hh24_17', 'hh24_18', 'hh24_19', 'hh24_20', 'hh24_21', 'hh24_22',
       'hh24_23', 'hh24_24', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12']
y_cols = ['elec']

In [ ]:
# train data temporalize
x = []
for num in tqdm(train['num'].unique()):
    num_x = train.loc[train['num'] == num, x_cols]
    for i in range(len(num_x) - timesteps + 1):
        x.append(num_x.iloc[i:i+timesteps])
X = np.array(x)
del x
np.save(path+'X_train_24.npy',X)
del X

# y = []
# for num in tqdm(train['num'].unique()):
#     num_y = train.loc[train['num'] == num, y_cols]
#     for i in range(len(num_y) - timesteps + 1):
#         y.append(num_y.iloc[i+timesteps-1])

# Y = np.array(y)
# del y, num_x, num_y
# np.save(path+'Y_train_24.npy',Y)
# del Y



100%|██████████| 323/323 [03:27<00:00,  1.56it/s]


In [ ]:
# test data temporalize
T = []
for num in tqdm(test['num'].unique()):
    num_x = test.loc[test['num'] == num, x_cols]
    for i in range(len(num_x) - timesteps + 1):
        T.append(num_x.iloc[i:i+timesteps])
T = np.array(T)

print(T.shape)

In [ ]:
np.save(path+'test_24.npy',T)

In [ ]:
# 다운로드 함수
import gdown, zipfile, os, glob, shutil
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

#위의 전처리된 3개 파일 다운로드 함수
download_file('1Jy25yk5tFvZnbVyZa5y5o9_IMsbw5GCC', '.X_train_24.npy')
download_file('1tzWAyQ6AjBGDvb9VkgKx8ELLKZbnbSZc', '.Y_train_24.npy')
download_file('1-4Z1eNwr1pumiHdHVT1hyqwlxVDUKYa_', '.test_24.npy')